In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('TFcalculationd2.csv')

In [3]:
dataset

,S.No,BP,AT,PM10,RH,PM2.5,WD,WS,NH3,CO,NO,NO2,NOx,SO2,Ozone,AQI,Label
0,1,739.927841,29.592495,444.669402,50.346059,155.981714,182.398863,1.685305,49.899503,1.667151,94.785703,76.884928,229.145198,23.380207,29.654703,1.309506,Moderate
1,2,739.927841,29.592495,444.669402,50.346059,155.981714,182.398863,1.685305,49.899503,1.667151,94.785703,76.884928,229.145198,23.380207,29.654703,2.197336,Moderate
2,3,739.927841,29.592495,444.669402,50.346059,155.981714,182.398863,1.685305,49.899503,1.667151,94.785703,76.884928,229.145198,23.380207,29.654703,2.197336,Moderate
3,4,739.927841,29.592495,444.669402,50.346059,155.981714,182.398863,1.685305,49.899503,1.667151,94.785703,76.884928,229.145198,23.380207,29.654703,2.197336,Moderate
4,5,739.927841,29.592495,444.669402,50.346059,155.981714,182.398863,1.685305,49.899503,1.667151,94.785703,76.884928,229.145198,23.380207,29.654703,2.197336,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,2492,972.180417,34.841250,112.833333,68.402083,41.104167,91.885417,2.437917,40.789167,1.719583,7.340833,26.114167,19.872500,6.555000,21.977046,1.254009,Moderate
2492,2493,972.026250,36.314167,288.927083,59.525833,75.822917,237.239583,0.865833,29.330000,2.177561,96.387500,44.670833,102.346250,8.974583,7.061667,1.368147,Moderate
2493,2494,971.963750,32.397500,83.135417,84.377083,44.541667,131.479167,0.590000,39.906704,2.177561,86.847517,71.109110,111.700845,10.585417,12.381667,2.007183,Moderate
2494,2495,972.887083,34.719167,312.250000,55.533750,75.531250,248.406250,0.470417,34.950417,2.177561,166.466563,86.191250,187.012119,21.204167,30.482500,1.472014,Moderate


In [4]:
dataset.columns

Index(['S.No', 'BP', 'AT', 'PM10', 'RH', 'PM2.5', 'WD', 'WS', 'NH3', 'CO',
       'NO', 'NO2', 'NOx', 'SO2', 'Ozone', 'AQI', 'Label'],
      dtype='object')

In [ ]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [ ]:
dataset

In [ ]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [ ]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'min_weight_fraction_leaf': [0.0, 0.05],
    'max_features': ['sqrt', 'log2'],
    'max_leaf_nodes': [None, 30, 50],
    'random_state': [42],
    'min_impurity_decrease': [0.0, 0.01],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.01],
    'monotonic_cst': [None]  # Example; specific use case dependent, often left out of generic grids
}
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:

re=grid.cv_results_
grid_predictions = grid.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [ ]:
# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [ ]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

In [ ]:
print("The confusion Matrix:\n",cm)

In [ ]:
print("The report:\n",clf_report)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

In [ ]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [ ]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [ ]:
table=pd.DataFrame.from_dict(re)

In [ ]:
table